### I. Chunk text

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# Đọc file và tách văn bản theo định dạng "số + ."
def read_and_chunk_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Tách văn bản dựa trên các chỉ mục "số + ."
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n", "."], chunk_size=200, chunk_overlap=30
    )
    chunks = text_splitter.split_text(content)
    return chunks

In [3]:
# Đọc file và chia thành các chunks
file_path = '../Knowledge_Graph/Data/content only ver4/visa1.1.txt'
chunks = read_and_chunk_file(file_path)

chunks

['1. Subclass 601\n2. Electronic Travel Authority\nUp to 3 months at a time\nThere is a service charge of AUD20 to use the Australian ETA app. There is no other charge payable for an ETA.',
 'In most cases you will be notified of the result of your ETA application immediately',
 '. Your application may take longer if you do not complete the application process correctly or we require further information. We recommend that you apply well ahead of when you want to travel.',
 'See all conditions\n3. With this visa you can\nTravel to Australia:',
 'Travel to Australia:\nYou can study or train for up to 3 months in total on this visa. If your main reason for travelling to Australia is to study, you should apply for\xa0a student visa.',
 'Note: You should apply for a Visitor (subclass 600) visa and complete the relevant health checks if you are seeking to:\nYou will need a visa if you enter Australia during a cruise.',
 'If your cruise is a round trip cruise starting from Australia, the time

In [4]:
# Sử dụng HuggingFaceEmbeddings để mã hóa các chunks
# Thay thế model embedding
embedding = HuggingFaceEmbeddings()
vector_store = Chroma.from_texts(chunks, embedding)

# Đặt câu hỏi và tìm các chunk có độ tương đồng cao nhất
query = "How long i can stay?"
docs_with_scores = vector_store.similarity_search_with_score(query, k=5)

# In ra top 5 chunk có độ tương đồng cao nhất
for doc, score in docs_with_scores:
    print(f"Chunk: {doc}\nScore: {score}\n")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8536\3961047193.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8536\3961047193.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


d:\Program\Py311\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


: 